# Using the ETL Process to Explore Effects of the COVID-19 Lockdowns on Selected Monthly Retail and Food Services Sales

Virginia Ogozalek

# Index

- [Abstract](#Abstract)
- [1. Introduction](#1.-Introduction)
- [2. Extract-Transform-Load](#2.-Extract-Transform-Load)
    - [2.1 The ETL Process](#2.1-The-ETL-Process)
    - [2.2 Data Exploration](#2.2-Data-Exploration)
    - [2.3 Data Preparation](#2.3-Data-Preparation)
    - [2.4 Read the Data Using Python](#2.4-Reading-the-Data-Using-Python)
         - [2.4.1 Reading Sample Data](#2.4.1-Reading-Sample-Data)
         - [2.4.2 Reading the MRST Data](#2.4.2-Reading-the-MRST-Data)
    - [2.5 Writing an Installation Script](#2.5-Writing-an-Installation-Script)
- [3. Analysis and Visualization](#3.-Project-Description)
    - [3.1 Running Queries in MySQL Workbech](#3.1-Running-Queries-in-MySQL-Workbech)
    - [3.2 Running Queries From Python](#3.2-Running-Queries-From-Python)
    - [3.3 Explore Trends](#3.3-Explore-Trends)
    - [3.4 Explore Percentage Change](#3.4-Explore-Percentage-Change)
    - [3.5 Explore Rolling Time Windows](#3.5-Explore-Rolling-Time-Windows)
- [Conclusion](#Conclusion)
- [References](#References)

[Back to top](#Index)


##  Abstract

This project uses the ETL process on the Monthly Retail Trade Survey (MRTS) dataset to explore how the COVID-19 lockdowns affected certain categories of retail and food services sales in the year 2020. After extracting, transforming, and loading the data into a MySQL database, queries are written to explore the data using trends, percentage change, and rolling time windows. The matplotlib library is used to create plots for visualization of the data. Conclusions reached based on data analysis is that sales in selected categories decreased during the COVID lockdown that started in March 2020 and then eventually started increasing again. 

[Back to top](#Index)


## 1. Introduction

Each month the United States Census Bureau produces monthly estimates of monthly sales and end-of-month inventories from data collected in the Monthly Retail Trade Survey (MRTS), based on reports from firms selected for the survey sample. This project will use the ETL process on the MRTS dataset to explore how the COVID-19 lockdowns affected certain categories of retail and food services sales in the year 2020. 

The first part of the project is the ETL process. Step 1 is to extract the data from the MRTS Excel spreadsheet to a CSV file, step 2 is to transform the data, and step 3 is to load it to a MySQL database.

The second part of the project is an exploration of the data through the process of writing queries, first using the MySQL Workbench and then in a Python environment. The matplotlib library will be used to create plots for visualization of the data. Queries will explore trends, percentage change, and rolling time windows.

The workflow model is shown below:

<img src="workflow_mod8.png" width="600" height="600">

The business categories that will be explored include the following: Full service restaurants; limited service eating places; beer, wine, and liquor stores; hobby, toy, and game stores; book stores; electronic shopping and mail-order houses; furniture and home furnishings stores; and total retail and food services sales.

These business categories are chosen based on anecdotal reports and personal experience suggesting that their sales were likely affected by the COVID-19 lockdowns.

[Back to top](#Index)

## 2. Extract-Transform-Load

The steps followed in this project are described below. Screenshots of code and program windows are included whenever possible.

[Back to top](#Index)

### 2.1 The ETL Process

The ETL process for this project is to create a data pipeline from the MRTS dataset to the applications where it can be analyzed to answer questions about how the COVID-19 lockdowns affected certain categories of retail and food services sales. 

The key steps to perform ETL on the provided MRTS dataset are:
1. Extract the data from the Excel spreadsheet *mrtssales92-present*.
2. Transform the data so it can be used to answer queries in SQL, particulary for queries about changes over time.
3. Load the data in mySQL and then into Python so plots can be created using Matplotlib.

Because the Python CSV library is used, boundaries between the Extract, Transform, and Load processes in this project are somewhat blurred.

[Back to top](#Index)

### 2.2 Data Exploration

The MRTS dataset contains monthly and yearly sales data for businesses that are categorized based on the 2012 North American Industry Classification System (NAICS). Businesses from a sample set are surveyed monthly, with a new sample set being chosen approximately every 5 years. The survey has been conducted since 1951, and the digital dataset goes back to 1992. 

After doing some research online about the MRTS dataset, what struck me as most interesting were 3 code links on kaggle.com, all of which referenced COVID-19. In this project, I explore how the COVID-19 lockdowns affected certain categories of retail and food services sales, with the dual goals of (1) learning how to use ETL to analyze the MRTS dataset and (2) satisfying my own curiosity about the effects of the pandemic on different areas of the economy.

Although a more up-to-date MRTS dataset is available, the dataset given for this project only includes data up to February, 2021.  

[Back to top](#Index)

### 2.3 Data Preparation

The process used for data preparation in this project is largely determined by my choice to use the CSV library to read the data into Python. I use the CSV library rather than pandas because that appears to be the recommendation for this project. The choice seems reasonable--a Google search suggests that CSV is faster for smaller (<1K) datasets, while pandas is faster for larger datasets.  Unlike pandas, however, which includes a *read_excel* function that directly interfaces with Excel, the CSV library only provides for reading existing CSV files. For this reason, it seems to make sense to prepare the data for CSV format when the data is still in Excel. For example, while the *read_excel* function in pandas provides for reading multiple worksheets into a single CSV file, using the CSV library seems to necessitate that this step be done by writing a Visual Basic program in Excel. 

When preparing the data for extraction from Excel, I tried to anticipate what form I would eventually need the data to be in. At this stage, my main goal was to avoid losing any data, in case it would be needed later on. In retrospect, this goal proved to be counterproductive. For example, I started copying data from all the different yearly worksheets into a single worksheet but soon stopped when I calculated how long that would take. Similarly, knowing that I would be analyzing trends over time, I started pivoting the Excel tables to place "Month/Year" as a column, but then stopped when I realized that the table I had in mind would have over 65 columns. For some reason, I did not consider selecting subsets of the different kinds-of-business columns or the different years worksheets or creating a different version of the SQL mrts table.

Preparations I did perform in Excel include the following:

* Remove rows of descriptive text at the top and bottom of each sheet
* Remove the adjusted data
* Remove column headers altogether to make eventually loading data into SQL easier (could also be accomplished by using *next( )* function in Python
* Change dates from number to text format
* Remove commas from numeric data
* Change missing data to zeros (I made this decision after spending a lot of time replacing missing data by using percentage formulas based on similar data--these calculations are saved and could be used in the future)

At the end of this process, I prepared 5 worksheets of data (years 2017 through 2021) for extraction as separate CSV files to Python and then SQL. None of the data has been lost for these five years. To extract the data as a CSV files, I selected *CSV UTF-8 (comma delimited)* file format from the Excel Save As command. The screenshot below shows part of the file *mrts2020.csv*.

<img src="screenshot1.png" width="600" height="600">

[Back to top](#Index)

### 2.4 Read the Data Using Python

Because the Python CSV library is used, the processes of reading the data using the Python CSV library and loading the data into SQL are combined. The code is shown in the rest of Section 2:

[Back to top](#Index)

### 2.4.1 Reading Sample Data

The sample data set I created to test my Python script is addams_family.csv, which is shown below:


<img src="screenshot2.png" width="600" height="600">


Here is the Python script for reading the file:


<img src="screenshot3.png" width="600" height="600">


Here is a screenshot of reading the data using the Terminal window.


<img src="screenshot4.png" width="600" height="600">


[Back to top](#Index)

### 2.4.2 Reading the MRTS Data

The steps for reading and printing the MRTS datafile are as follows:
1. Open the MRTS CSV file.
2. Use the csv_reader function from the CSV library to read the data, using commas as delimiters.
3. Loop through the data and print the output, one row at a time.

Here are the lines of code used to read and print out the MRTS CSV file for 2020 using a Python script.

<img src="screenshot5.png" width="600" height="600">

The code for the other years is the same except for the csv file names.

[Back to top](#Index)

### 2.5 Writing an Installation Script

Here is the Python installation script used to load the MRTS data into SQL. The first section of code shown below imports the necessary libraries for the script, makes the connection to the MySQL server using a YAML file for added security, and creates the MySQL database "mrts" if it does not already exist.

<img src="screenshot6.png" width="600" height="600">

The next part of the installation script creates the MySQL table "mrts2020" if it does not already exist. Each row represents a business category from the MRTS dataset, and each column represents sales for a month in that year. Other columns include the NAICS code, the kind of business, and the total yearly sales for the business category. 

Note that the file reading process for the CSV file is embedded in the code for loading the data into SQL. The first 4 lines of that section check to make sure that data has not already been entered into the table. Then the MTRS CSV file is read in one row at a time and each comma-separated value is loaded into the SQL mrts database using an INSERT INTO query as shown. When the process is complete, the connection to the MySQL service is closed.

<img src="screenshot7.png" width="600" height="600">

The same process is repeated to create the tables "mrts2021," "mrts2019," "mrts2018," and "mrts2017." The code for each is the same except that the names of the CSV files and the MySQL tables are changed to correspond to each year.


[Back to top](#Index)

## 3. Analysis and Visualization

The queries I run against the MRTS dataset are designed to explore effects of the COVID-19 lockdowns on various kinds of businesses in the MRTS dataset. Each query is run against the MRTS dataset using both the MySQL Workbench and a Python environment.

The differences between the two approaches is that in the MySQL Workbench, the query code can be both edited and run in the Workbench environment, while code written in the Python environment is run from the Terminal window. The advantage of running queries in the MySQL Workbench is that the graphical user interface makes writing and testing queries easy, while the main disadvantage of the MySQL Workbench is a lack of resources for creating data visualizations. The Python environment, on the other hand, is rich in available libraries for data visualization, but the queries must be created by first writing a Python script and then running the script from a Terminal window.

[Back to top](#Index)

### 3.1 Running Queries in MySQL Workbench

Here are two queries run against the MRTS dataset in MySQL Workbench to verify that everything works as expected.

1. SELECT COUNT(*) FROM mrts2020

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The result is 65, which matches the number of rows in the table.

2. SELECT business, jan, feb, mar, apr, may, jun, jul aug, sep, oct, nov, dece

	&nbsp;&nbsp;&nbsp;&nbsp;FROM mrts2020
    
	&nbsp;&nbsp;&nbsp;&nbsp;WHERE business = 'Full service restaurants' OR
    
    &nbsp;&nbsp;&nbsp;&nbsp;business = 'Limited service eating places'
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The results are the two rows of the monthly data for these two business. Checking against the CSV file shows that the data is correct.

[Back to top](#Index)

### 3.2 Running Queries From Python

Python scripts can be used to test the previous queries on the MRTS dataset. The first section of code shown below imports the necessary libraries for the script and makes the connection to the MySQL server using a YAML file for added security. The same SQL query executed first in the MySQL Workbench is pasted into the code, with the results assigned to the variable *sql*. Then the result is assigned to the variable *count* and printed out. 

<img src="screenshot8.png" width="600" height="600">

The script is run from the Terminal window and the result of 65 is shown, which matches the result in the MySQL Workbench.

<img src="screenshot9.png" width="1000" height="600">

The second query can also be added to the Python script as shown below:

<img src="screenshot10.png" width="800" height="1000">

When the script is run from the Terminal window, the result is the same as in the MySQL Workbench.

<img src="screenshot11.png" width="1500" height="600">

[Back to top](#Index)

### 3.3 Explore Trends

An economic trend is the direction in which data is moving. Economic trends are considered important measures for predicting quantities because people believe that economic data from the past can be used to predict the future, even though the forecasts are not that reliable. Nevertheless, businesses and other institutions need to plan for the future, and they need something that will tell them what the economic future might look like.

In this section, I look at the MTRS dataset with a focus on the relationship between the sales data and the COVID-19 lockdowns. It is my general hypothesis that COVID caused a disruption in economic trends, and I will look at specific kinds of businesses to see what happened. As of December 2022, it seems like COVID is still having a world-wide economic impact. As far as using economic trends to try to predict the future, it will probably take several years worth of new data to even start beginning to understand the economic effects of COVID.

While economic trends are described by numeric data, visualizations are key to identifying what the data reveals. All queries in this section will be made in the Python environment because visualization options are limited in MySQL Workbench, while the Python environment provides access to libraries such as Matplotlib and Seaborn, which provide a wide range of visualization options.

### 3.3.1 Query 1: How were sales at full- and limited-service eating places affected by COVID?

For the purposes of this project, I am associating the category "full-service eating places" with eat-in dining and "limited-service" with fast food and takeout. My hypothesis is that while sales in both categories decreased, sales in full-service eating places were affected much more than sales in limited-service eating places.

The code for getting the numeric data for both of these kinds of businesses is already shown in the section above. Adding the code below plots both sets of data on a Matplotlib line graph:

<img src="screenshot13.png" width="1000" height="1000">

<img src="screenshot12.png" width="600" height="600">

Just looking at this graph brings back memories of February and March 2020, when COVID hit. At first, everone was locked down pretty much completely, with minimal excursions out to seek food and other necessities. Gradually, people started going out again, but dine-in restaurants sales remained substantially lower than takeout.

The graph can be extended back another year to explore the situation further. The code below plots both sets of data for 2019 and 2020 on a similar Matplotlib line graph.

<img src="screenshot14.png" width="800" height="1000">

<img src="screenshot15.png" width="1000" height="400">

The plot shows that while both full and limited service eating places had similar sales before the start of the pandemic, during the rest of 2020 limited-service restaurants returned to their previous level of sales while full service restaurants never did. It will probably take several years worth of new data to really understand what COVID did to both types of food businesses.


### 3.3.2 Query 2: How were sales of businesses selling products related to staying at home affected by COVID?

In this query, I look at trends in business categories related to activities done when staying at home: "Beer, wine, and liquor stores"; "Hobby, toy, and game stores"; and "Book stores." My hypothesis is that sales in all of these areas increased, maybe with beer, wine, and liquor sales increasing the most.

The code for getting the numeric data for these three kinds of businesses and for plotting both sets of data on a Matplotlib line graph is shown below:

<img src="screenshot16.png" width="1000" height="1000">

Here is the plot:

<img src="screenshot17.png" width="1000" height="600">

This graph suggests that sales in both the "Beer, wine, and liquor stores" and "Hobby, toy, and game stores" categories are seasonal, with sharp increases occurring during the Thanksgiving and Christmas seasons (more so for beer, wine, and liquor stores). However, after Christmas, there are differences in the sales trends for each category in 2019 and even more extreme differences in 2020 when COVID hits. 

In 2019, beer, wine, and liquor store store sales dip very slightly in January, then start rising gradually through the summer months, then increase in the fall before the Christmas season surge hits again. In 2020, however, beer, wine, and liquor store sales increased fairly sharply in February, when rumors of the imminent COVID pandemic were beginning to swirl--perhaps people were stocking up? When the lockdowns were put in place in March, sales decreased slightly, then started an increasing trend over the summer and into the fall until the seasonal uptick for Christmas season 2020. Overall, beer, wine, and liquor store sales were greater in 2020 than in 2019.  

On the other hand, hobby, toy, and game store sales, as well as book store sales descreased when COVID hit. This seems surprising, since anecdotal reports as well as personal experience suggested that while people were spending so much time at home, many of them were enjoying their hobbies, having family game nights, and reading books. An explanation for this unexpected result may be that the operative word is "store"--people may have been spending more money on hobbies, toys, games, and books, but they were buying them online at sites like Amazon.com, not in stores. Since there is no comparable online system for buying beer, wine, and liquor, people still had to buy these items at a store, either in person or through a delivery service. 


[Back to top](#Index)

### 3.4 Explore Percentage Change

A percentage change in economics is the percent by which a quantity increases or decreases from one point in time to another. Percentage change can be helpful because it is relative, not just showing the amount of change, but telling how much the quantity increased or decreased relative to what it was before. 

The business categories explored in this section are (1) electronic shopping and mail-order sales and (2) new and used car sales. The exploration for the new and used car sales is a substitute for the suggested exploration of women's clothing and men's clothing businesses, since some data for men's clothing stores was missing in 2020.  

### 3.4.1 Query 3: How were electronic shopping and mail-order sales affected by COVID?

In this query, I look at trends in the category "Electronic shopping and mail-order sales." My hypothesis is that percentage increases will be greater in 2020 when in COVID hits than during the same time period in 2019.

Because I was querying about changes over time, I decided to reformat my data in SQL so that "month" was a column. To do this, I adapted code from StackOverflow on how to convert a row to a column to meet my needs for this query (https://stackoverflow.com/questions/13944417/mysql-convert-column-to-row-pivot-table).

The code for getting the numeric data and percentage changes and for plotting the data on a Matplotlib line graph is shown below:

<img src="screenshot18a.png" width="800" height="1000">
<img src="screenshot18b.png" width="800" height="1000">
<img src="screenshot18c.png" width="800" height="1000">

To label the percentage change at each point on the line graph, I adapted code provided in this article: https://queirozf.com/entries/add-labels-and-text-to-matplotlib-plots-annotation-examples. Here is the plot:

<img src="screenshot19.png" width="1000" height="400">

The two greatest percentage increases other than Christmas season sales are from Febuary 2020 to March 2020 and from March 2020 to April of 2020, as hypothesized. I wonder how much of that was toilet paper?

### 3.4.2 Query 4: How were new and used car sales affected by COVID?


This query is a substitution for the suggested percentage query about men's clothing and women's clothing, since some of the 2020 data for men's clothing is missing. New car sales and used car sales are similar categories, since combined they represent all car sales.

The code for getting the numeric data and calculating percentages for each sale category as part of the whole, as well as for plotting the data on a Matplotlib line graph, is shown below:

<img src="screenshot24.png" width="800" height="1000">

Here is the plot:

<img src="screenshot25.png" width="400" height="400">

The percentages of sales remain fairly constant throughout the year, with the percentage of new car sales increasing and the percentage of used car sales decreasing in March, when the lockdowns were in full force. What seems more striking than any time-related trends is that new car sales are such a large percentage of all car sales.

To get a little bit more insight into what was happening during COVID, I look at sales amounts directly. The code I use is the same as the code in Query 1, substituting new car sales and used car sales as the business categories. The plot is shown below:

<img src="screenshot26.png" width="400" height="400">

Looking at the actually sales data shows that car sales dropped significantly when COVID hit, a trend that is lost when only percentages are considered. The relationship between percentages of new and used car sales in March is likely explained by the appearance that new cars sales seem to drop more precipitously than used car sales in March when the COVID lockdowns were at their height. 


[Back to top](#Index)

### 3.5 Explore Rolling Time Windows

A rolling time window in economics divides the times series data into subsets equal-size windows that shift forward with the passage of time. Aggregate operations can be performed on the data in these windows.

For both of the rolling time window queries, I reformatted my data in SQL so that "month" is a column. To write my rolling time window code for this project, I adapted the code in this article: https://medium.com/an-idea/rolling-sum-and-average-window-functions-mysql-7509d1d576e6.

The business categories explored in this section are (1) furniture and home furnishings store sales and (2) total retail and food services sales.

### 3.5.1 Query 5: How were furniture and home furnishings store sales affected by COVID?

In this query, I look at trends in the category "Furniture and home furnishings store." My hypothesis is that sales will decrease, with a lag behind the onset of COVID because the plot shows a 3-month rolling time window average.

The code for getting the numeric data and percentage changes and for plotting the data on a Matplotlib line graph is shown below:

<img src="screenshot20.png" width="800" height="1000">
<img src="screenshot20a.png" width="800" height="1000">

Here is the plot:

<img src="screenshot21.png" width="1000" height="400">

The sales decrease markedly starting in March and reach their lowest point in May. This makes sense because data from the two preceding months are always included in the average for each rolling time window.

### 3.5.2 Query 6: How were total retail and food services sales affected by COVID?

In this query, I look at trends in the category "Total retail and food services sales." My hypothesis is that sales will decrease, with a lag behind the onset of COVID because the plot shows a 3-month rolling time window average.

The code for getting the numeric data and percentage changes and for plotting the data on a Matplotlib line graph is shown below:

<img src="screenshot22a.png" width="800" height="1000">
<img src="screenshot22b.png" width="800" height="1000">

Here is the plot:

<img src="screenshot23.png" width="1000" height="600">

Just as with the furniture sales, the total sales decrease markedly starting in March, but unlike the furniture sales, they reach their lowest point in April and are already increasing by May. I find this surprising. To get a little more insight, I look at total sales for 2019 and 2020, using the same code as Query 1, substituting "Total retail and food services sales" for the business category.

<img src="screenshot27.png" width="1000" height="600">

It appears from this graph that claims about a second economic recovery after COVID are true.

[Back to top](#Index)

## Conclusion

My conclusions are that most of my hypotheses about the effects of the COVID-19 lockdowns on sales in different business categories were shown to be on target when the MRTS data was visualized. In all business categories, there was a significant dip in sales in March, when COVID hit and the lockdowns went into effect.

What is surprising to me is how quickly the economy, as evidenced by this sales data, seemed to recover. The "Total retail and food services sales" data was especially encouraging. I look forward to working with an updated MRTS dataset at some point, maybe a few years from now when the effects of COVID will become more evident.

On a coding note, this project was quite a learning experience. As I worked on the project, I felt like I had painted myself into a corner by using the CSV library instead of pandas. Now that I am reflecting on the experience, I would say that I gained invaluable experience in MySQL problem solving skills, as well as the hard-earned insight that I could have written code in the Python environment to transform my CSV-based data to pandas dataframes.

In summation, in the office hours for Module 8, Kloe Ng repeated a quote about the CSV library: "Pandas is primarily used for data analysis and data science, so using it for working with CSVs seems like using a sledgehammer on a nail."

After completing this project primarily using the CSV library, I would like to offer my own observation: "Using the CSV library when Pandas is available is like a surgeon choosing to perform heart surgery with mittens on."

Fortunately, in this project, I think the patient survives.




Note: The sledgehammer quote is from:
https://www.blog.pythonlibrary.org/2022/11/03/python-101-how-to-work-with-csv-files/


[Back to top](#Index
)
## References


Davidson, Adam and Alex Blumberg. "Can Economic Trends Predict the Future?" *Planet Money*. NPR, 2010. https://www.npr.org/templates/story/story.php?storyId=122142337

Landlord. "USA Monthly Retail Sales: Estimate of Monthly Retail and Food Services Sales with NAICS Codes." *Kaggle*, 2020. (https://www.kaggle.com/datasets/landlord/usa-monthly-retail-trade/code)

Otwell, Joshua. "Rolling sum and average — Window Functions MySQL." *An Idea (by Ingenious Piece)*, 2020. https://medium.com/an-idea/rolling-sum-and-average-window-functions-mysql-7509d1d576e6

Python Library. "Python 101 - How to Work with CSV files." *Mouse vs Python*, 2022. https://www.blog.pythonlibrary.org/2022/11/03/python-101-how-to-work-with-csv-files/  

Queirozf.com. "Add Labels and Text to Matplotlib Plots: Annotation Examples." Queirozf.com, 2022. https://queirozf.com/entries/add-labels-and-text-to-matplotlib-plots-annotation-examples

United States Census Bureau. "Monthly Retail Trade Survey: About the Survey." https://www.census.gov/retail/mrts/about_the_surveys.html
